In [1]:
import cv2
import tkinter as tk
import numpy as np
from tkinter import filedialog, Toplevel, Canvas, Button, Frame
from PIL import Image, ImageTk
import os

In [11]:
path_img = input("Digite o caminho da pasta com o vídeo: ").replace('\\', '/') + '/'
video = path_img + input("Digite o nome do arquivo de vídeo: ")
frames_folder = input("Digite o caminho da pasta para onde os frames salvos irão: ").replace('\\', '/') + '/'
path = os.getcwd() + '/'

Digite o caminho da pasta com o vídeo:  C:\Users\PC\Documents\documents\ifam\TETI\Listas_Atividades\Trabalho_Manipulacao_Videos\teti_imagens
Digite o nome do arquivo de vídeo:  mont_russa.mp4
Digite o caminho da pasta para onde os frames salvos irão:  C:\Users\PC\Documents\documents\ifam\TETI\Listas_Atividades\Trabalho_Manipulacao_Videos\teti_imagens\frames


In [12]:
k1 = np.array([[-1, -1, -1],[-1, 9, -1],[-1, -1, -1]])
k2 = np.array([[1, 0, 0],[0, 0, 0],[0, 0, -1]])
k3 = np.array([[0, 0, 0],[0, 1, 0],[0, 0, 0]])
k4 = np.array([[0, -1, 0],[-1, 5, -1],[0, -1, 0]])
k_blur = np.array([[1/9, 1/9, 1/9],[1/9, 1/9, 1/9],[1/9, 1/9, 1/9]], dtype=np.float32)
k_sharpen = np.array([[0, -1, 0],[-1, 5, -1],[0, -1, 0]], dtype=np.float32)
k_emboss = np.array([[2, 1, 0],[1, 0, -1],[0, -1, -2]], dtype=np.float32)
k_laplace = np.array([[0,1,0],[1,-4,1],[0,1,0]], dtype = np.float32)
k_sobel_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]], dtype=np.float32)
k_sobel_y = np.array([[1, 2, 1],[0, 0, 0],[-1, -2, -1]], dtype=np.float32)

In [13]:
def modo_def(frame, modo):
    if modo == 'gray':
         frame_alterado = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif modo == 'binary':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, frame_alterado = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
    elif modo == 'laplace':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_alterado = cv2.filter2D(gray, -1, k_laplace)
    elif modo == 'blur':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_alterado = cv2.filter2D(gray, -1, k_blur)
    elif modo == 'sharpen':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_alterado = cv2.filter2D(gray, -1, k_sharpen)
    elif modo == 'emboss':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_alterado = cv2.filter2D(gray, -1, k_emboss)
    elif modo == 'edges':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_alterado = cv2.Canny(gray, 100, 200)
    elif modo == 'sobel-x':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_alterado = cv2.filter2D(gray, -1, k_sobel_x)
    elif modo == 'sobel-y':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_alterado = cv2.filter2D(gray, -1, k_sobel_y)
    elif modo == 'zoom':
        frame_alterado = frame[200:550,200:700]
    elif modo == 'lines':
        frame_alterado = cv2.filter2D(frame, -1, k2)
    elif modo == 'relevo':
        frame_alterado = cv2.filter2D(frame, -1, k1)
    elif modo == 'filtro3':
        frame_alterado = cv2.filter2D(frame, -1, k3)
    elif modo == 'filtro4':
        frame_alterado = cv2.filter2D(frame, -1, k4)
    else:
        frame_alterado = frame

    return frame_alterado

In [14]:
def key_def(key, modo, tempo):
    if key == ord('g'):
        modo = 'gray'
    elif key == ord('b'):
        modo = 'binary'
    elif key == ord('1'):
        modo = 'color'
    elif key == ord('l'):
        modo = 'laplace'
    elif key == ord('z'):
        modo = 'zoom'
    elif key == ord('p'):
        modo = 'lines'
    elif key == ord('r'):
        modo = 'relevo'
    elif key == ord('m'):
        modo = 'filtro3'
    elif key == ord('n'):
        modo = 'filtro4'
    elif key == ord('u'):
        modo = 'blur'
    elif key == ord('o'):
        modo = 'emboss'
    elif key == ord('h'):
        modo = 'sharpen'
    elif key == ord('x'):
        modo = 'sobel-x'
    elif key == ord('y'):
        modo = 'sobel-y'
    elif key == ord('e'):
        modo = 'edges'
    elif key == ord('+'):
        tempo-=10
        if tempo <=0:
            tempo=1
    elif key == ord('-'):
        tempo+=10

    return(modo, tempo)

In [15]:
def salvar_frames(cap, frame_inicial, frame_final, posicao, frame_alterado):
    if frame_inicial is not None and frame_final is not None:
        for i in range(frame_inicial, frame_final + 1):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame_to_save = cap.read()
            if ret:
                os.chdir(frames_folder)
                cv2.imwrite(f'frame_{i}.png', frame_to_save)
                os.chdir(path)                
    else:
        os.chdir(frames_folder)
        cv2.imwrite(f'frame_{posicao}.png', frame_alterado)
        os.chdir(path)

In [16]:
def registrar_pontos_de_corte(cap, modo, tempo):
    pontos_de_corte = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_alterado = modo_def(frame, modo)
        cv2.imshow('Frame', frame_alterado)
        key = cv2.waitKey(tempo) & 0xFF
        if key == ord(' '):  # Barra de espaço
            pontos_de_corte.append(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000)  # Registra o tempo em segundos
            print(f"Ponto de corte registrado em {pontos_de_corte[-1]} segundos")
        elif key == ord('q'):
            break
    return pontos_de_corte

def gerar_videos_intervalos(video, pontos_de_corte, modo, tempo):
    cap = cv2.VideoCapture(video)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    pontos_de_corte.sort()
    pontos_de_corte.append(total_frames / fps)  # Adiciona o tempo final do vídeo
    for i in range(len(pontos_de_corte) - 1):
        inicio = pontos_de_corte[i]
        fim = pontos_de_corte[i + 1]
        cap.set(cv2.CAP_PROP_POS_MSEC, inicio * 1000)
        frames_intervalo = []
        while True:
            ret, frame = cap.read()
            if not ret or cap.get(cv2.CAP_PROP_POS_MSEC) / 1000 >= fim:
                break
            frame_alterado = modo_def(frame, modo)
            frames_intervalo.append(frame_alterado)
        out = cv2.VideoWriter(f'video_{i+1}.mp4', fourcc, fps, (width, height))
        for frame in frames_intervalo:
            out.write(frame)
        out.release()
    cap.release()
    cv2.destroyAllWindows()

def salvar_frames_intervalos(video, pontos_de_corte, modo, tempo):
    cap = cv2.VideoCapture(video)
    pontos_de_corte.sort()
    pontos_de_corte.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))  # Adiciona o frame final do vídeo
    for i in range(len(pontos_de_corte) - 1):
        inicio = pontos_de_corte[i]
        fim = pontos_de_corte[i + 1]
        cap.set(cv2.CAP_PROP_POS_FRAMES, inicio)
        frames_intervalo = []
        while True:
            ret, frame = cap.read()
            if not ret or cap.get(cv2.CAP_PROP_POS_FRAMES) >= fim:
                break
            frame_alterado = modo_def(frame, modo)
            frames_intervalo.append(frame_alterado)
        for j, frame in enumerate(frames_intervalo):
            cv2.imwrite(f'frame_{i+1}_{j}.png', frame)
    cap.release()
    cv2.destroyAllWindows()

def escolher_salvar_frames_ou_video(video, pontos_de_corte, modo, tempo):
    print("Escolha uma opção:")
    print("1. Salvar frames em separado")
    print("2. Converter em um novo vídeo")
    escolha = input("Digite a opção: ")
    if escolha == "1":
        salvar_frames_intervalos(video, pontos_de_corte, modo, tempo)
    elif escolha == "2":
        gerar_videos_intervalos(video, pontos_de_corte, modo, tempo)
    else:
        print("Opção inválida")

In [17]:
import cv2

def play_video_reverse(video, modo, tempo):
    frames = []
    cap_ = cv2.VideoCapture(video)
    
    # Ler todos os frames do vídeo e armazená-los
    while True:
        ret, frame = cap_.read()
        if not ret:
            break
        frame_alterado = modo_def(frame, modo)
        frames.append(frame_alterado)

    # Verifica se há frames para reproduzir
    if not frames:
        print("Nenhum frame foi lido do vídeo.")
        cap_.release()  # Libera o objeto de captura antes de retornar
        return

    print(f"Total de frames lidos: {len(frames)}")  # Para verificar quantos frames foram lidos

    # Loop para reproduzir os frames na ordem inversa até que 'q' seja pressionado
    try:
        while True:
            index = len(frames) - 1  # Começa do último frame
            while index >= 0:
                cv2.imshow('Video Reverse', frames[index])
                if cv2.waitKey(tempo) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()  # Fecha todas as janelas
                    return  # Sai da função
                index -= 1  # Decrementa o índice para ir para o frame anterior
            
            # Opcional: adicionar uma pausa entre as repetições
            cv2.waitKey(1000)  # Pausa de 1 segundo entre repetições, se desejado
    finally:
        cap_.release()  # Garante que o objeto de captura seja liberado ao final

# Exemplo de uso
# video_path = 'seu_video.mp4'  # Substitua pelo caminho do seu vídeo
# modo = ...  # Defina o modo desejado
# tempo = 100  # Defina um tempo de espera em milissegundos
# play_video_reverse(video_path, modo, tempo)

In [19]:
modo = 'color'
tempo = 10
posicao = 0
posicao_fixo = 0
pause = 0
frame_inicial = None
frame_final = None

cap = cv2.VideoCapture(video)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)

if not os.path.exists(frames_folder):
    os.makedirs(frames_folder)
while True:
    if not pause:
        posicao += 1
        ret, frame = cap.read()
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            posicao=0
            continue

    frame_alterado = modo_def(frame, modo)
    if frame_alterado is not None:
        cv2.imshow('Frame', frame_alterado)
    else:
        print("Frame alterado é None")

    key = cv2.waitKey(tempo) & 0xFF

    modo, tempo = key_def(key, modo, tempo)
    
    if key == ord('p'):
        pause = not pause
    elif key == ord('s'):
       salvar_frames(cap, frame_inicial, frame_final, posicao, frame_alterado)
    elif key == ord('i'):
        frame_inicial = posicao
        print(f'Frame inicial marcado: {frame_inicial}')
    elif key == ord('f'):
        frame_final = posicao
        print(f'Frame final marcado: {frame_final}')
    elif key == ord('c'):  # Registrar pontos de corte
        pontos_de_corte = registrar_pontos_de_corte(cap, modo, tempo)
        escolher_salvar_frames_ou_video(video, pontos_de_corte, modo, tempo)
    elif key == ord('='): 
        print("Reproduzindo vídeo em reversa...")
        play_video_reverse(video, modo, tempo)
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

188
Escolha uma opção:
1. Salvar frames em separado
2. Converter em um novo vídeo


KeyboardInterrupt: Interrupted by user

In [2]:
def reproduzir_video_reverso():
    global video_capture, canvas_video
    frames = []
    cap_ = video_capture
    
    # Ler todos os frames do vídeo e armazená-los
    while True:
        ret, frame = cap_.read()
        if not ret:
            break
        frames.append(frame)

    # Verifica se há frames para reproduzir
    if not frames:
        print("Nenhum frame foi lido do vídeo.")
        cap_.release()  # Libera o objeto de captura antes de retornar
        return

    print(f"Total de frames lidos: {len(frames)}")  # Para verificar quantos frames foram lidos

    # Criar uma nova janela para reproduzir o vídeo reverso
    janela_video_reverso = Toplevel()
    janela_video_reverso.title("Reproduzir Vídeo Reverso")

    # Canvas para exibir os frames do vídeo
    canvas_video = Canvas(janela_video_reverso, width=800, height=600, bg="black")
    canvas_video.pack()

    # Função interna para reproduzir os frames em ordem reversa
    def reproduzir_frames_reverso(index):
        global frame_atual
        if index < 0:
            index = len(frames) - 1  # Reinicia a reprodução
        frame = frames[index]
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_atual = ImageTk.PhotoImage(Image.fromarray(frame_rgb))
    
        # Atualiza o Canvas
        canvas_video.create_image(0, 0, anchor="nw", image=frame_atual)
        canvas_video.img = frame_atual  # Previne garbage collection
    
        # Continuar reprodução
        canvas_video.after(tempo_atual, reproduzir_frames_reverso, index - 1)
    
    # Função para iniciar a reprodução
    def iniciar_reproducao():
        reproduzir_frames_reverso(len(frames) - 1)

    # Fechar a janela e liberar o vídeo ao fechar
    def fechar_video_reverso():
        if cap_ and cap_.isOpened():
            cap_.release()
        janela_video_reverso.destroy()

    janela_video_reverso.protocol("WM_DELETE_WINDOW", fechar_video_reverso)
    
    # Botões de controle
    Button(janela_video_reverso, text="Reproduzir", command=iniciar_reproducao).pack(side="left", padx=5)
    Button(janela_video_reverso, text="Capturar Quadro", command=capturar_quadro).pack(side="left", padx=5)
    Button(janela_video_reverso, text="Fechar", command=fechar_video_reverso).pack(side="right", padx=5)